# ASBE - Automatic Stopping for Batch Experiments

> API details.

In [ ]:
from nbdev import *

In [ ]:
#default_exp core

In [ ]:
#export
import numpy as np
from asbe.base import *
from modAL.models.base import BaseLearner
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from typing import Union, Optional, Callable
from copy import deepcopy
from pylift.eval import UpliftEval

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
#export
def random_batch_sampling(classifier, X_pool, n2, **kwargs):
    "Randomly sample a batch from a pool of unlabaled samples"
    n_samples = X_pool.shape[0]
    query_idx = np.random.choice(range(n_samples), size=n2,replace=False)
    return X_pool[query_idx], query_idx

def uncertainty_batch_sampling(classifier, X_pool, n2, **kwargs):
    "Select the top $n_2$ most uncertain units"
    ite_preds, y1_preds, y_preds = classifier.predict(X_pool, **kwargs)
    # Calculate variance based on predicted
    if y1_preds.shape[0] <= 1 or \
    len(y1_preds.shape) <= 1:
            raise Exception("Not possible to calculate uncertainty when dimensions <=1 ")
    ite_vars = np.var(classifier.estimator.y1_preds - classifier.estimator.y0_preds, axis=1)
    query_idx = np.argsort(ite_vars)[-n2:][::-1]
        
    return X_pool[query_idx], query_idx

def type_s_batch_sampling(classifier, X_pool, n2, **kwargs):
    "Select highest type-s"
    ite_preds, y1_preds, y_preds = classifier.predict(X_pool, **kwargs)
    prob_s = np.sum(ite_preds > 0, axis=1)/ite_preds.shape[1]
    prob_s_sel = np.where(prob_s > 0.5, 1-prob_s, prob_s) + .0001
    query_idx = np.argsort(prob_s_sel)[-n2:][::-1]
    
    return X_pool[query_idx], query_idx
   

def expected_model_change_maximization(classifier, X_pool, n2, **kwargs):
    """
    Implementation of EMCM for ITE - using a surrogate SGD model.
    """
    # Get mean of the trained prediction
    ite_train_preds, y1_train_preds, y0_train_preds = \
        classifier.predict(classifier.X_training, **kwargs)
    if ite_train_preds.shape[1] < 1:
        raise ValueError("The treatment effect does not have uncertainty around it - \
                         consider using a different estimator")
    # Get mean of predicted ITE
    ite_pool_preds, y1_pool_preds, y0_pool_preds = \
        classifier.predict(X_pool, **kwargs)
    # Then scale the data so sgd works the best
    sc = StandardScaler()
    X_scaled = sc.fit_transform(classifier.X_training)
    # Fit approx model
    # calc type-s error
    train_type_s_prob_1 = np.sum(ite_train_preds > 0, axis=1)/ite_train_preds.shape[1]
    train_type_s = np.where(train_type_s_prob_1 > 0.5, 1-train_type_s_prob_1, train_type_s_prob_1) + .0001
    pool_type_s_prob_1 = np.sum(ite_pool_preds > 0, axis=1)/ite_pool_preds.shape[1]
    pool_type_s = np.where(pool_type_s_prob_1 > 0.5, 1-pool_type_s_prob_1, pool_type_s_prob_1) + .0001
    classifier.approx_model.fit(
        X = X_scaled,
        y = np.mean(ite_train_preds, axis=1),
        sample_weight = 5*train_type_s)
    # Using list as it is faster than appending to np array
    query_idx = []
    # Using a loop for the combinatorial opt. part
    for ix in range(n2):
        if n2 > (X_pool.shape[0]):
            raise IndexError("Too many samples are queried from the pool ($n_2 > ||X_pool||$)")
        # Select randomly from X_pool
        prob_sampling = np.ones((X_pool.shape[0]))/(X_pool.shape[0]-len(query_idx))
        # Set the probability of already selected samples to 0
        if ix > 0:
            prob_sampling[query_idx] = 0
        # B = 100 by default, can be modified by kwargs
        considered_ixes = np.random.choice(X_pool.shape[0],
                                         size = kwargs["B"] if "B" in kwargs else 100,
                                         replace=False, 
                                         p=prob_sampling)
        # Calculate the grads for all the 
        grads = np.array([])
        for considered_ix in considered_ixes:
            new_X = sc.transform(X_pool[considered_ix].reshape(1, -1))
            app_predicted_ite = classifier.approx_model.predict(new_X)
            # bootstrapping accroding to eq. 11 of Cai and Zhang
            true_ite = np.random.choice(ite_pool_preds[considered_ix],
                                        size=kwargs["K"] if "K" in kwargs else 5)
            grad = np.sum(np.abs(np.kron((true_ite - app_predicted_ite),new_X)))
            grads = np.append(grads, grad)
        if np.max(grads) < kwargs["threshold"] if "threshold" in kwargs else 0:
            break
        classifier.model_change = np.append(classifier.model_change,np.max(grads))
        query_idx.append(int(considered_ixes[np.argmax(grads)]))
        classifier.approx_model.partial_fit(
            sc.transform(X_pool[int(query_idx[ix])].reshape(1, -1)),
            np.random.choice(ite_pool_preds[int(query_idx[ix])], size=1),
            sample_weight = np.array(pool_type_s[int(query_idx[ix])]).ravel())
        
    return X_pool[query_idx], query_idx

In [ ]:
# %nbdev_export
# def variance_assignment(classifier, X_pool, n2, **kwargs):
#     """Function to assign treatment or control based on the variance of the cf
#     """
#     ite_pool_preds, y1_pool_preds, y0_pool_preds = \
#         classifier.predict(X_pool, **kwargs)
#     var_y1 = np.var(y1_pool_preds, axis=1)
#     var_y0 = np.var(y0_pool_preds, axis=1)
#     prob_of_treatment = var_y1/(var_y1+var_y0)
#     drawn_treatment = np.random.binomial(1, prob_of_treatment)
#     return drawn_treatment

In [ ]:
#export
estimator_type = ClassifierMixin
class ASLearner(BaseLearner):
    """A(ctively)S(topping)Learner class for automatic stopping in batch-mode AL"""
    def __init__(self,
                 estimator: estimator_type=None, 
                 query_strategy=None,
                 assignment_fc=None,
                 X_training: np.ndarray = None,
                 t_training: np.ndarray = None,
                 y_training: np.ndarray = None,
                 X_pool: np.ndarray = None,
                 X_test: np.ndarray = None,
                 approx_model: RegressorMixin = None
                ) -> None:
        self.estimator = estimator
        self.query_strategy = query_strategy
        self.assignment_fc = assignment_fc
        self.X_training = X_training
        self.y_training = y_training
        self.t_training = t_training
        self.X_pool     = X_pool
        self.X_test     = X_test
        self.approx_model = approx_model
        self.model_change = np.array([])
        
    def _add_queried_data_class(self, X, t, y):
        self.X_training = np.vstack((self.X_training, X))
        self.t_training = np.concatenate((self.t_training, t))
        self.y_training = np.concatenate((self.y_training, y))
    
    def _update_estimator_values(self):
        self.estimator.__dict__.update(X_training = self.X_training,
                               y_training  =        self.y_training,
                               t_training  =        self.t_training,
                               X_test      =        self.X_test)

    def teach(self, X_new, t_new, y_new, **kwargs):
        """Teaching new instances to the estimator selected bu the query_strategy
        
        If no `assignment_fc` is added, all selected samples are used
        If assignment function is added, only those instances are used, where
        $\hat{T} = T$
        """
        if self.assignment_fc is not None:
            X_new, t_new, y_new = self.assignment_fc(
                self, X_new, t_new,
                y_new, simulated=kwargs["simulated"] if "simulated" in kwargs else False)
        else:
            try:
                y_new = np.take_along_axis(y_new, t_new[:, None], axis=1)
            except:
                pass
        self._add_queried_data_class(X_new, t_new.ravel(), y_new.ravel())
        self.fit()

    def fit(self):
        self._update_estimator_values()
        self.estimator.fit()
        
    def predict(self, X=None, **kwargs):
        """Method for predicting treatment effects within Active Learning
        
        Default is to predict on the unlabeled pool"""
        if X is None:
            raise Exception("You need to supply an unlabeled pool of instances (with shape (-1,{}))".format(self.X_training.shape[1]))
        self.preds = self.estimator.predict(X, **kwargs)
        return self.preds
    
    def score(self, preds=None, y_true=None, t_true=None, metric = "Qini"):
        """
        Scoring the predictions - either ITE or observed outcomes are needed.
        
        If observed outcomes are provided, the accompanying treatments are also needed.
        """
        if metric not in ["Qini", "PEHE", "Cgains"]:
            raise ValueError(f"Please use a valid error (PEHE, Qini, Cgains), {metric} is not valid")
        if metric == "Qini":
            upev = UpliftEval(t_true, y_true, self.preds[0] if preds is None else preds)
            self.scores = upev
            vscore = self.scores.q1_aqini
        if metric == "PEHE":
            vscore = np.sqrt(np.mean(np.square(preds - y_true)))
        if metric == "Cgains":
            upev = UpliftEval(t_true, y_true, self.preds[0] if preds is None else preds)
            self.scores = upev
            vscore = self.scores.cgains
        return vscore

In [ ]:
#export
class ITEEstimator(BaseEstimator):
    """ Class for building a naive estimator for ITE estimation
    """
    def __init__(self,
                 model: estimator_type = None,
                 two_model: bool = False,
                 ps: Callable = None,
                 **kwargs
                ) -> None:
        self.model = model
        self.two_model = two_model
        self.ps_model = ps
        
    def _fit_ps_model(self):
        if self.ps_model is not None:
            self.ps_model.fit(self.X_training, self.t_training)

    def fit(self,X_training: np.ndarray = None,
                 t_training: np.ndarray = None,
                 y_training: np.ndarray = None,
                 X_test: np.ndarray = None,
                 ps_scores: np.ndarray = None):
            
        if X_training is not None:
            self.X_training = X_training
            self.y_training = y_training
            self.t_training = t_training
            self.X_test = X_test
        self.N_training = self.X_training.shape[0]
        try:
            self._fit_ps_model()
            ps_scores = self.ps_model.predict_proba(self.X_training)
        except:
            ps_scores = None
            # if "N_training" not in self.__dict__:
        #     self.N_training = self.X_training.shape[0]
        if ps_scores is not None:
            X_to_train_on = np.hstack((self.X_training, ps_scores[:,1].reshape((-1, 1))))
        else:
            X_to_train_on = self.X_training                
        if self.two_model:
            if hasattr(self, "m1") is False:
                self.m1 = deepcopy(self.model)
            control_ix = np.where(self.t_training == 0)[0]
            self.model.fit(X_to_train_on[control_ix,:],
                           self.y_training[control_ix])
            self.m1.fit(X_to_train_on[-control_ix,:],
                        self.y_training[-control_ix])
        else:
            self.model.fit(np.hstack((X_to_train_on,
                                      self.t_training.reshape((self.N_training, -1)))),
                           self.y_training)
            
    def _predict_without_proba(self, model, X, **kwargs):
        return model.predict(X,
            return_mean = kwargs["return_mean"] if "return_mean" in kwargs else True)
    
    def _fix_dim_pred(self, preds):
        pred_length = preds.shape[0]
        if preds.shape[1] == 1:
            if np.all(preds == 0):
                preds = np.hstack((preds, np.ones(pred_length).reshape((-1,1))))
            elif np.all(preds == 1): 
                preds = np.hstack((preds, np.zeros(pred_length).reshape((-1,1))))
            preds = preds[:, ]
        return preds         
    
    def predict(self, X=None, **kwargs):
        if X is None:
            X = self.X_test
        if self.ps_model is not None and self.ps_model.coef_ is not None:
            pred_ps_scores = self.ps_model.predict_proba(X)[:, 1]
            X = np.hstack((X, pred_ps_scores.reshape(-1, 1)))
        N_test = X.shape[0]
        try:
            if self.two_model:
                self.y1_preds = self.m1.predict_proba(X)
                self.y0_preds = self.model.predict_proba(X)
            else:
                self.y1_preds = self.model.predict_proba(
                                    np.hstack((X,
                                    np.ones(N_test).reshape(-1,1))))
                self.y0_preds = self.model.predict_proba(
                    np.hstack((X,
                               np.zeros(N_test).reshape(-1,1))))
            self.y1_preds = self._fix_dim_pred( self.y1_preds)
            self.y0_preds = self._fix_dim_pred( self.y0_preds)
        except AttributeError:
            try:
                if self.two_model:
                    self.y1_preds = self._predict_without_proba(self.m1, X, **kwargs)
                    self.y0_preds = self._predict_without_proba(self.model, X, **kwargs)
                else: 
                    self.y1_preds = self._predict_without_proba(self.model,
                             np.hstack((X,
                             np.ones(N_test).reshape(-1,1))), **kwargs)
                    self.y0_preds = self._predict_without_proba(self.model,
                        np.hstack((X,
                                   np.zeros(N_test).reshape(-1,1))), **kwargs)
            except:
                raise AttributeError("No method found for predicting with the supplied class")
        return self.y1_preds - self.y0_preds, self.y1_preds, self.y0_preds

In [ ]:
X = np.random.normal(size = 1000).reshape((500,2))
t = np.random.binomial(n = 1, p = 0.5, size = 500)
y = np.random.binomial(n = 1, p = 1/(1+np.exp(X[:, 1]*2 + t*3)))
X_test = np.random.normal(size = 200).reshape((100,2))
t_test = np.random.binomial(n = 1, p = 0.5, size = 100)
y_test = np.random.binomial(n = 1, p = 1/(1+np.exp(X_test[:, 1]*2 + t_test*3)))
a = ITEEstimator(LogisticRegression(solver="lbfgs"), two_model = True, ps=LogisticRegression())
a.fit(X, t, y)
assert type(a.model) == LogisticRegression  # test assigning a model
assert a.X_training.shape  == (500,2)       # test data passing for class
assert a.model.intercept_ is not None

In [ ]:
np.unique(y).shape[0]

2

In [ ]:
# a = ITEEstimator(RandomForestClassifier(), X, t, y, X_test, two_model = False )

In [ ]:
#export
def variance_based_assf(classifier, X, t, y, simulated=False):
    ite_preds, y1_preds, y0_preds = classifier.predict(X, return_mean=False)
    if len(y1_preds.shape) <= 1:
            raise ValueError("Not possible to calculate variance with dim {}".format(y1_preds.shape))
    prop_score = np.var(y1_preds,axis=1)/(
        np.var(y1_preds, axis=1)+np.var(y0_preds,axis=1))
    t_assigned = np.random.binomial(1, prop_score)
    if simulated:
        try:
            y = np.take_along_axis(y, t_assigned[:, None], axis=1)
            t = t_assigned
            usable_units = np.repeat(True, repeats=X.shape[0])
        except:
            raise ValueError("Potential outcomes are needed in a matrix with shape (n,2)")
    else:
        usable_units = np.where(t_assigned == t)
    return X[usable_units], t[usable_units], y[usable_units]

In [ ]:
from xbart import XBART

In [ ]:
ds = {"X_training":X,
     "y_training": y,
     "t_training": t,
     "X_pool": deepcopy(X_test), 
     "y_pool": deepcopy(y_test),
     "t_pool": deepcopy(t_test),
     "X_test": X_test,
     "y_test": y_test,
      "t_test": t_test
     }

In [ ]:
asl = BaseActiveLearner(estimator = BaseITEEstimator(model = XBART()), 
                        acquisition_function=BaseAcquisitionFunction(),
                        assignment_function=variance_based_assf,
                        stopping_function = None,
                        dataset=ds)
asl.fit()
ite_pred = asl.predict(asl.dataset["X_test"])
X_sel, query_sel = asl.query(asl.dataset["X_test"])
asl.teach(X_sel, t_test[query_sel], y_test[query_sel])
assert ite_pred.shape[0] == 100
assert X_sel.shape       == (10,2)

(100,)


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
ite_pred

In [ ]:
np.random.seed(1005)
n_train = 100
p       = 5
n_pool  = 1000
n_test  = 1000
n2      = 5
X_train = np.random.normal(size = n_train*p).reshape((n_train,p))
t_train = np.random.binomial(n = 1, p = 0.5, size = n_train)
y_train = np.random.binomial(n = 1, p = 1/(1+np.exp(-1*(X_train[:, 1]*2 + t_train*3))))

X_pool = np.random.normal(size = n_pool*p).reshape((n_pool,p))
t_pool = np.random.binomial(n = 1, p = 0.5, size = n_pool)
y_pool = np.random.binomial(n = 1, p = 1/(1+np.exp(-1*(X_pool[:, 1]*2 + t_pool*3))))

X_test = np.random.normal(size = n_test*p).reshape((n_test,p))
t_test = np.random.binomial(n = 1, p = 0.5, size = n_test)
y_test = np.random.binomial(n = 1, p = 1/(1+np.exp(-1*(X_test[:, 1]*2 + t_test*3))))


In [ ]:
asl = ASLearner(estimator = ITEEstimator(model = XBART(),two_model=False), 
         query_strategy=expected_model_change_maximization,
         X_training = X_train,
         t_training = t_train,
         y_training = y_train,
         X_pool     = X_pool,
         X_test     = X_test,
         approx_model=SGDRegressor())
asl.fit()
p_ite, p_y1, p_y0 = asl.predict(asl.X_test, return_mean=False)
print("Qini before AL: {}".format(asl.score(preds=np.mean(p_ite, axis=1),
                                            y_true=y_test, t_true=t_test)))
qini_vals = []
for batch_round in range(21):
    X_query, ix = asl.query(asl.X_pool, n2=n2, return_mean=False)
    asl.teach(X_query, t_pool[ix], y_pool[ix])
    asl.X_pool = np.delete(asl.X_pool,ix, axis=0)
    t_pool     = np.delete(t_pool,ix, axis=0) 
    y_pool     = np.delete(y_pool,ix, axis=0) 
    p_ite, p_y1, p_y0 = asl.predict(asl.X_test, return_mean=False)
    qini_vals.append(asl.score(preds=np.mean(p_ite, axis=1), y_true=y_test, t_true=t_test))
    if batch_round % 5 == 0:
        print("Qini after round {} of AL: {}".format(batch_round,qini_vals[batch_round]))